In [1]:
import pandas as pd
import pandas as pd
import torch
from datetime import datetime

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback

from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)






2024-06-30 14:21:23.072350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 14:21:23.072448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 14:21:23.216249: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [22]:



df_train = pd.read_csv(r'/kaggle/input/sample-3000/train_df_2100_manual_final.csv')
df_val = pd.read_csv(r'/kaggle/input/sample-3000/validation_df_450_manual_final.csv')
df_test = pd.read_csv(r'/kaggle/input/sample-3000/test_df_450_manual_final.csv')



total_df = pd.concat([df_train, df_val, df_test], axis=0)
total_df.shape

df_train.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)
df_val.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)
df_test.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)
total_df.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)


df_train['manual_labeling_new'] = df_train['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})
df_test['manual_labeling_new'] = df_test['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})
df_val['manual_labeling_new'] = df_val['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})

total_df['manual_labeling_new'] = total_df['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})


In [4]:
pip install spacy

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [8]:
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 73.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [9]:
import spacy
from transformers import MarianMTModel, MarianTokenizer
import random

# Load spaCy's Spanish language model
nlp = spacy.load("es_core_news_sm")

# Synonym Replacement using spaCy
def get_spacy_synonyms(word):
    synonyms = set()
    for syn in nlp.vocab[word].cluster:
        synonyms.add(syn.text)
    return list(synonyms)

def synonym_replacement(sentence, n):
    doc = nlp(sentence)
    words = [token.text for token in doc]
    new_words = words.copy()
    random_word_list = list(set([word for word in words if nlp.vocab[word].has_vector]))  # Words with vectors
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_spacy_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    sentence = ' '.join(new_words)
    return sentence

# Back Translation
def back_translation(text, src_lang="es", mid_lang="en"):
    src_to_mid = f'Helsinki-NLP/opus-mt-{src_lang}-{mid_lang}'
    mid_to_src = f'Helsinki-NLP/opus-mt-{mid_lang}-{src_lang}'
    
    tokenizer_src_to_mid = MarianTokenizer.from_pretrained(src_to_mid)
    model_src_to_mid = MarianMTModel.from_pretrained(src_to_mid)
    
    tokenizer_mid_to_src = MarianTokenizer.from_pretrained(mid_to_src)
    model_mid_to_src = MarianMTModel.from_pretrained(mid_to_src)
    
    translated = model_src_to_mid.generate(**tokenizer_src_to_mid(text, return_tensors="pt", padding=True))
    translated_text = tokenizer_src_to_mid.decode(translated[0], skip_special_tokens=True)
    
    back_translated = model_mid_to_src.generate(**tokenizer_mid_to_src(translated_text, return_tensors="pt", padding=True))
    back_translated_text = tokenizer_mid_to_src.decode(back_translated[0], skip_special_tokens=True)
    
    return back_translated_text

# Random Insertion
def random_insertion(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    for _ in range(n):
        add_word = words[random.randint(0, len(words)-1)]
        new_words.insert(random.randint(0, len(new_words)), add_word)
    return ' '.join(new_words)

# Random Deletion
def random_deletion(sentence, p):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    new_words = [word for word in words if random.uniform(0, 1) > p]
    if len(new_words) == 0:
        return words[random.randint(0, len(words)-1)]
    return ' '.join(new_words)

# Random Swap
def random_swap(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
    return ' '.join(new_words)


In [13]:
def augment_text(sentence):
    # Apply synonym replacement
    sentence = synonym_replacement(sentence, 2)
    
#     # Apply back translation
#     sentence = back_translation(sentence)
    
    # Apply random insertion
    sentence = random_insertion(sentence, 2)
    
    # Apply random deletion
    sentence = random_deletion(sentence, 0.1)
    
    # Apply random swap
    sentence = random_swap(sentence, 2)
    
    return sentence


In [23]:
# Apply augmentation to increase training set size

train_texts = df_train['comment_processed_text']

train_labels = df_train['manual_labeling_new'] 
augmented_texts = train_texts.apply(augment_text)

# Combine original and augmented training data
train_texts_augmented = pd.concat([train_texts, augmented_texts])
train_labels_augmented = pd.concat([train_labels, train_labels])

In [24]:
from sklearn.utils import shuffle

train_texts_augmented, train_labels_augmented = shuffle(train_texts_augmented, train_labels_augmented)

# Create a DataFrame with the augmented texts and labels
augmented_df = pd.DataFrame({
    'text': train_texts_augmented,
    'label': train_labels_augmented
})

# Check the distribution of each class
class_distribution = augmented_df['label'].value_counts()
print(class_distribution)


label
0    1686
1    1288
2    1226
Name: count, dtype: int64


In [26]:
train_texts_augmented.shape

(4200,)

In [27]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

In [28]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',  # Padding
            truncation=True,       # Truncation
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)  # Ensure labels are tensors of type long
        }


In [29]:
# Extract texts and labels


# train_texts = df_train['comment_processed_text'].tolist()
# train_labels = df_train['manual_labeling_new'].tolist()
train_texts = train_texts_augmented.tolist()
train_labels = train_labels_augmented.tolist()

val_texts = df_val['comment_processed_text'].tolist()
val_labels = df_val['manual_labeling_new'].tolist()
test_texts = df_test['comment_processed_text'].tolist()
test_labels = df_test['manual_labeling_new'].tolist()

In [30]:
MAX_LEN = 512

# Create train, validation, and test datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, MAX_LEN)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer, MAX_LEN)

In [31]:
train_dataset[0]


{'text': 'si nuevas ideas es funcionarios vertedero basura de gobiernos ex partidos gobernantes ya dejen de hablar de que los gobiernos pasados este gobierno tiene amigos asesores el y es la miasma que los los ya pasados',
 'input_ids': tensor([    4,  1096,  3390,  5404,  1028,  5095,  4329,  8806,  6351,  1009,
          4545,  1158,  4945, 23564,  1319,  9139,  1009,  1942,  1009,  1041,
          1067,  4545, 17664,  1277,  2022,  1394,  2745, 17089,  1039,  1040,
          1028,  1032, 14861, 30958,  1141,  1041,  1067,  1067,  1319, 17664,
             5,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,   

In [35]:
import os
# Ensure the output directory exists
output_dir = './results_beto'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Ensure the logging directory exists
logging_dir = './logs'
if not os.path.exists(logging_dir):
    os.makedirs(logging_dir)

In [36]:

model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=3)


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install accelerate -U

In [ ]:
pip show accelerate


In [ ]:
#pip install transformers[torch]

In [37]:
# Define custom metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:


# Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,  # Use the output directory variable
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logging_dir,
    report_to=[],
)

In [ ]:
# Data Collator
data_collator = DataCollatorWithPadding(tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Add custom metrics
    data_collator=data_collator       # Add data collator
)

In [38]:
# Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,  # Use the output directory variable
    num_train_epochs=10,  # Increase number of epochs
    per_device_train_batch_size=16,  # Adjust batch size
    per_device_eval_batch_size=16,
    learning_rate=2e-5,  # Adjust learning rate
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logging_dir,
    eval_strategy="epoch",  # Evaluate at each epoch
    save_strategy="epoch",  # Save at each epoch
    save_total_limit=3,  # Only keep the last 3 checkpoints
    load_best_model_at_end=True,  # Load the best model found during training at the end
    metric_for_best_model="accuracy",  # Use accuracy to determine the best model
    greater_is_better=True,
    report_to=[],  # Disable all integrations
)

In [39]:
# Data Collator
data_collator = DataCollatorWithPadding(tokenizer)
# Trainer with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Add custom metrics
    data_collator=data_collator,       # Add data collator
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Add early stopping
)


In [40]:
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.778503,0.653333,0.629851,0.648355,0.653333
2,No log,0.659725,0.706667,0.697168,0.699088,0.706667
3,No log,0.718265,0.708889,0.712121,0.727311,0.708889
4,0.607400,0.991410,0.662222,0.656785,0.683109,0.662222
5,0.607400,1.082442,0.715556,0.719882,0.732194,0.715556
6,0.607400,1.419932,0.724444,0.718615,0.717830,0.724444
7,0.607400,1.635005,0.726667,0.726689,0.726723,0.726667
8,0.080100,1.825081,0.728889,0.731123,0.734475,0.728889
9,0.080100,1.854122,0.731111,0.732155,0.733399,0.731111
10,0.080100,1.877511,0.735556,0.735784,0.736025,0.735556


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=1320, training_loss=0.26216962734858196, metrics={'train_runtime': 2160.2092, 'train_samples_per_second': 19.443, 'train_steps_per_second': 0.611, 'total_flos': 1.1050763544576e+16, 'train_loss': 0.26216962734858196, 'epoch': 10.0})

In [42]:
# Evaluate the model on the validation set
val_results = trainer.evaluate()
print("Validation Results:", val_results)

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results:", test_results)



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Validation Results: {'eval_loss': 1.8775105476379395, 'eval_accuracy': 0.7355555555555555, 'eval_f1': 0.7357841803793918, 'eval_precision': 0.7360245957080489, 'eval_recall': 0.7355555555555555, 'eval_runtime': 8.0397, 'eval_samples_per_second': 55.972, 'eval_steps_per_second': 1.866, 'epoch': 10.0}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Test Results: {'eval_loss': 1.5375936031341553, 'eval_accuracy': 0.7622222222222222, 'eval_f1': 0.7581489162178818, 'eval_precision': 0.7588333088631136, 'eval_recall': 0.7622222222222222, 'eval_runtime': 8.0461, 'eval_samples_per_second': 55.928, 'eval_steps_per_second': 1.864, 'epoch': 10.0}


In [43]:
val_results

{'eval_loss': 1.8775105476379395,
 'eval_accuracy': 0.7355555555555555,
 'eval_f1': 0.7357841803793918,
 'eval_precision': 0.7360245957080489,
 'eval_recall': 0.7355555555555555,
 'eval_runtime': 8.0397,
 'eval_samples_per_second': 55.972,
 'eval_steps_per_second': 1.866,
 'epoch': 10.0}

In [44]:
test_results

{'eval_loss': 1.5375936031341553,
 'eval_accuracy': 0.7622222222222222,
 'eval_f1': 0.7581489162178818,
 'eval_precision': 0.7588333088631136,
 'eval_recall': 0.7622222222222222,
 'eval_runtime': 8.0461,
 'eval_samples_per_second': 55.928,
 'eval_steps_per_second': 1.864,
 'epoch': 10.0}

In [45]:
# Generate and print the classification report for validation set
preds_val = trainer.predict(val_dataset)
labels_val = preds_val.label_ids
predictions_val = preds_val.predictions.argmax(-1)
print("\nClassification Report (Validation):")
print(classification_report(labels_val, predictions_val, target_names=['negative', 'neutral', 'positive']))

# Generate and print the classification report for test set
preds_test = trainer.predict(test_dataset)
labels_test = preds_test.label_ids
predictions_test = preds_test.predictions.argmax(-1)
print("\nClassification Report (Test):")
print(classification_report(labels_test, predictions_test, target_names=['negative', 'neutral', 'positive']))

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



Classification Report (Validation):
              precision    recall  f1-score   support

    negative       0.78      0.78      0.78       181
     neutral       0.60      0.61      0.61       138
    positive       0.82      0.81      0.81       131

    accuracy                           0.74       450
   macro avg       0.73      0.73      0.73       450
weighted avg       0.74      0.74      0.74       450



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



Classification Report (Test):
              precision    recall  f1-score   support

    negative       0.78      0.85      0.81       181
     neutral       0.71      0.59      0.65       138
    positive       0.78      0.82      0.80       131

    accuracy                           0.76       450
   macro avg       0.76      0.75      0.75       450
weighted avg       0.76      0.76      0.76       450



In [46]:
# Define the paths where you want to save the model and tokenizer
import os

model_save_path = './saved_model'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

tokenizer_save_path = './saved_tokenizer'
if not os.path.exists(tokenizer_save_path):
    os.makedirs(tokenizer_save_path)

    



In [47]:
# Save the model
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)

('./saved_tokenizer/tokenizer_config.json',
 './saved_tokenizer/special_tokens_map.json',
 './saved_tokenizer/vocab.txt',
 './saved_tokenizer/added_tokens.json')